In [1]:
from collections import defaultdict
from IPython.display import display
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import nltk.stem.wordnet as wordnet
import numpy as np
import pandas as pd
import random
import sklearn
import string
import time

<b>Requires grants_gov.csv, research_grant_history.csv, and nsf_grants.csv</b>

In [2]:
sw = set(stopwords.words('english'))
wnl = wordnet.WordNetLemmatizer()

In [3]:
pd.set_option('max_columns', 40)
grants_df = pd.read_csv('../scraped_data/grants_gov.csv', delimiter="~")
grants_df.head()

,OpportunityID,OpportunityTitle,OpportunityNumber,OpportunityCategory,FundingInstrumentType,CategoryOfFundingActivity,CategoryExplanation,CFDANumbers,EligibleApplicants,AdditionalInformationOnEligibility,AgencyCode,AgencyName,PostDate,CloseDate,LastUpdatedDate,AwardCeiling,AwardFloor,EstimatedTotalProgramFunding,ExpectedNumberOfAwards,Description,Version,CostSharingOrMatchingRequirement,ArchiveDate,GrantorContactEmail,GrantorContactEmailDescription,GrantorContactText,AdditionalInformationURL,AdditionalInformationText,OpportunityCategoryExplanation,CloseDateExplanation
0,262148,Establishment of the Edmund S. Muskie Graduate...,SCAPPD-14-AW-161-SCA-08152014,D,CA,O,Public Diplomacy,19.040,25,Eligibility for U.S. institutions is limited t...,DOS-SA,Bureau of South and Central Asian Affairs,8152014,9042014.0,8202014,600000.0,400000.0,600000.0,1.0,The Office of Press and Public Diplomacy of th...,Synopsis 2,No,10042014.0,DelageEM@state.gov,Inquiries,"Ellen Delage, Bureau of South and Central Asia...",NaN,NaN,NaN,NaN
1,262149,Eradication of Yellow Crazy Ants on Johnston A...,F14AS00402,D,CA,"AG,ENV,NR",NaN,15.608,99,The recipient has already been selected for th...,DOI-FWS,Fish and Wildlife Service,8152014,8222014.0,8152014,0.0,0.0,0.0,NaN,Funds under this award are to be used for the ...,Synopsis 1,No,8232014.0,Connie_Sauer@fws.gov,Connie_Sauer@fws.gov,"Connie Sauer, 503-490-0827&lt;br/&gt;Connie_Sa...",http://www.grants.gov/,http://www.grants.gov/,NaN,NaN
2,131073,"Cooperative Ecosystem Studies Unit, Piedmont S...",G12AS20003,D,CA,ST,NaN,15.808,25,This financial assistance opportunity is being...,DOI-USGS1,Geological Survey,11172011,11292011.0,11282011,0.0,0.0,31900.0,1.0,The USGS Southeast Ecological Science Center s...,Synopsis 2,No,12172011.0,fgraves@usgs.gov,fgraves@usgs.gov,"Faith Graves, 703-648-7356&lt;br/&gt;fgraves@u...",http://www.grants.gov/,http://www.grants.gov/,NaN,NaN
3,196613,OVW FY 2012 Training and Technical Assistance ...,OVW-2012-3380,D,CA,LJL,NaN,16.526,"06,12",NaN,USDOJ-OJP-OVW,Office on Violence Against Women,8292012,10152012.0,8292012,2500000.0,NaN,2500000.0,1.0,There are different statutory and regulatory r...,Synopsis 1,No,11142012.0,Christina.Murray@usdoj.gov,Christina.Murray@usdoj.gov,Christina Murray &lt;br/&gt;Phone 202-307-0344,http://www.ovw.usdoj.gov/docs/grantee-reportin...,OVW FY 2012 Training and Technical Assistance ...,CategoryExplanation,NaN
4,65553,Affordable Care Act (ACA): Childhood Obesity R...,RFA-DP-11-007,D,CA,HL,NaN,93.535,99,NaN,HHS-CDC-HHSCDCERA,Centers for Disease Control and Prevention - ERA,1192011,4252011.0,4152011,5250000.0,0.0,NaN,4.0,This Funding Opportunity Announcement (FOA) so...,Synopsis 10,No,5182011.0,pgotim@cdc.gov,General Email,Centers for Disease Control and Prevention (CD...,Please click on Full Announcement at the top o...,Please click on Full Announcement at the top o...,NaN,On-time submission requires that electronic ap...


In [4]:
faculty_df = pd.read_csv('../scraped_data/faculty_profiles.csv', delimiter="~")
faculty_df.head()

,faculty_name,faculty_profile_url,l_expertise,department,title_name,faculty_site_url,lab_url,faculty_email,description,description_links,article_date_1,title_of_news_1,link_to_news_1,description_teaser_1,article_date_2,title_of_news_2,link_to_news_2,description_teaser_2,article_date_3,title_of_news_3,link_to_news_3,description_teaser_3,article_date_4,title_of_news_4,link_to_news_4,description_teaser_4,article_date_5,title_of_news_5,link_to_news_5,description_teaser_5,first_name,last_name,full_name
0,David A. Aaker,/faculty/david-aaker,"business,marketing,branding",Haas School of Business,Professor of Marketing and Public Policy,http://www.haas.berkeley.edu/faculty/aaker.html,http://groups.haas.berkeley.edu/marketing/,aaker@haas.berkeley.edu,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,david,aaker,david aaker
1,Pieter Abbeel,/faculty/pieter-abbeel,"robotics,machine learning",Division of Computer Science/EECS,Professor,http://www.cs.berkeley.edu/~pabbeel,NaN,pabbeel@cs.berkeley.edu,Robotics and Machine Learning.,NaN,"February 22, 2016",“Deep Learning”: A Giant Step for Robots,/bakarfellows/profile/pieter_abbeel,Bakar Fellow Pieter Abbeel studies deep lear...,"February 19, 2016",Three young faculty members honored by White ...,/news/three-young-faculty-members-honored-whit...,Three UC Berkeley faculty members named as r...,"May 21, 2015",New ‘deep learning’ technique enables robot m...,/news/new-deep-learning-technique-enables-robo...,UC Berkeley researchers have developed algor...,"December 17, 2012",Big NSF grant funds research into training ro...,/news/big-nsf-grant-funds-research-training-ro...,"What if robots and humans, working together,...","August 23, 2011",UC Berkeley robotics expert named among world...,/news/uc-berkeley-robotics-expert-named-among-...,"Pieter Abbeel, a UC Berkeley, professor know...",pieter,abbeel,pieter abbeel
2,Elizabeth Abel,/faculty/elizabeth-abel,"feminist theory,psychoanalysis,virginia woolf,...",Department of English,Professor of English,http://english.berkeley.edu/profiles/5,NaN,eabel@uclink.berkeley.edu,Elizabeth Abel's general research interest is...,NaN,"February 15, 2011","Jim Crow signs as symbols of subjugation, tro...",/news/jim-crow-signs-symbols-subjugation-troph...,"In the mid 1960s, landmark laws brought an o...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,elizabeth,abel,elizabeth abel
3,Dor Abrahamson,/faculty/dor-abrahamson,"mathematical cognition,design-based research,m...",Graduate School of Education,Associate Professor of Cognition and Development,http://gse.berkeley.edu/people/dor-abrahamson,http://edrl.berkeley.edu/,dor@berkeley.edu,Dor Abrahamson studies the process of mathema...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,dor,abrahamson,dor abrahamson
4,Norman Abrahamson,/faculty/norman-abrahamson,"civil and environmental engineering,earthquake...",Department of Civil and Environmental Engineering,Adjunct Professor of Civil and Environmental E...,http://www.ce.berkeley.edu/faculty/faculty.php...,NaN,naa3@earthlink.net,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,norman,abrahamson,norman abrahamson


In [7]:
grant_history_df = pd.read_csv('../scraped_data/research_grant_history.csv', delimiter=",")
grant_history_df.head()

/home/jerry/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (18,19) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Activity Type,Amount,Sponsor Class,Sponsor,Division,Department,Fund,UCB Award Number,PI Name,Project Begin Date,Project End Date,Title,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19
0,Applied research,"$179,032",State of California,California Department of Health Care Services,School of Public Health,NaN,15952.0,021331-002,"Colford Jr, John M",7/1/2006,9/30/2006,DNS AIDS Training,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Basic research,"$154,578",State of California,California Department of Social Services,School of Social Welfare,NaN,15959.0,021362-002,"Needell, Barbara",7/1/2006,9/30/2006,Performance Indicators/California Children's S...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Instruction,"$225,000",State of California,California Department of Social Services,School of Social Welfare,Social Welfare,15960.0,021363-002,"Midgley, James",7/1/2006,9/30/2006,Title IV-E Social Work Training Program,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Instruction,"$47,138",State of California,California Department of Social Services,School of Social Welfare,Social Welfare,15960.0,021363-002,"Midgley, James",7/1/2006,9/30/2006,Title IV-E Social Work Training Program,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Basic research,"$65,000",Federal,NIH National Institutes of Health - Miscellaneous,VC Res Other Research Units,The California Institute for Quantitative Bios...,78561.0,021425-002,"Keasling, Jay",7/1/2006,12/31/2006,Model-Driven Strain Engineering for Isoprenoid...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
nsf_grants_df = pd.read_csv('../scraped_data/nsf_grants.csv', delimiter="~")
nsf_grants_df.head()

,Headline,Due Date Start,Due Date End,Description,Link
0,Resource Implementations for Data Intensive Re...,None,2017-02-27,As part of NSF’s Cyberinfrastructure Framework...,https://nsf.gov/funding/pgm_summ.jsp?pims_id=5...
1,Computer Science for All,None,2017-02-28,This program aims to provide all U.S.students ...,https://nsf.gov/funding/pgm_summ.jsp?pims_id=5...
2,EPSCoR Research Infrastructure Improvement Tra...,None,2017-02-28,The Experimental Program to Stimulate Competit...,https://nsf.gov/funding/pgm_summ.jsp?pims_id=5...
3,Industry-University Cooperative Research Cente...,None,2017-02-28,The Industry-University Cooperative Research C...,https://nsf.gov/funding/pgm_summ.jsp?pims_id=5...
4,SOCIOLOGY PROGRAM - Doctoral Dissertation Rese...,None,2017-02-28,The Sociology Program supports basic research ...,https://nsf.gov/funding/pgm_summ.jsp?pims_id=5...


In [9]:
descriptions = grants_df['Description']

In [10]:
punc_trans = str.maketrans(string.punctuation, " " * len(string.punctuation))
num_trans = str.maketrans('', '', '123456789')

In [11]:
# Prepare grants.gov grants
cleaned = []
for desc in descriptions:
    try:
        desc = desc.lower()
        desc = desc.translate(punc_trans)
        desc = desc.translate(num_trans)
        first = filter(lambda x: x.lower() not in sw, desc.split())
        second = [wnl.lemmatize(x) for x in first]
        cleaned.append(second)
    except:
        cleaned.append(["Nothing"])
    

In [12]:
print("Example of cleaned grants synopsis: ")
print(cleaned[0])

Example of cleaned grants synopsis: 
['office', 'press', 'public', 'diplomacy', 'bureau', 'south', 'central', 'asian', 'affair', 'u', 'department', 'state', 'coordination', 'bureau', 'european', 'eurasian', 'affair', 'bureau', 'education', 'cultural', 'affair', 'announces', 'open', 'competition', 'cooperative', 'agreement', 'implement', 'edmund', 'muskie', 'graduate', 'internship', 'program', 'herein', 'referred', 'muskie', 'internship', 'program', 'muskie', 'internship', 'program', 'designed', 'promote', 'mutual', 'understanding', 'foster', 'democratization', 'transition', 'market', 'economy', 'eurasia', 'intensive', 'professional', 'education', 'program', '0', 'muskie', 'intern', 'begin', 'four', 'month', 'professional', 'internship', 'united', 'state', 'immediately', 'upon', 'completion', 'fulbright', 'foreign', 'student', 'program', 'internship', 'provide', 'opportunity', 'professional', 'development', 'well', 'reinforce', 'skill', 'gained', 'academic', 'study']


In [13]:
corpus = [' '.join(doc) for doc in cleaned]

In [14]:
grants_vectorizer = TfidfVectorizer()
grants_matrix = grants_vectorizer.fit_transform(corpus)

In [23]:
print("Number of grants.gov grants: " + str(grants_matrix.shape[0]))
print("Size of vocabulary: " + str(len(grants_vectorizer.vocabulary_)))

Number of grants.gov grants: 41536
Size of vocabulary: 48164


In [16]:
# Prepare NSF grants
cleaned = []
nsf_descriptions = nsf_grants_df['Description']
    
for desc in nsf_descriptions:
    try:
        desc = desc.lower()
        desc = desc.translate(punc_trans)
        desc = desc.translate(num_trans)
        first = filter(lambda x: x.lower() not in sw, desc.split())
        second = [wnl.lemmatize(x) for x in first]
        cleaned.append(second)
    except:
        cleaned.append(["Nothing"])
        
nsf_corpus = [' '.join(doc) for doc in cleaned]

In [17]:
nsf_vectorizer = TfidfVectorizer(min_df=4, vocabulary=grants_vectorizer.vocabulary_)
nsf_matrix = nsf_vectorizer.fit_transform(nsf_corpus)

In [18]:
print("Number of NSF grants: " + str(nsf_matrix.shape[0]))

Number of NSF grants: 484


Create faculty profiles. Currently takes all past research from research_grant_history.csv and vectorizes using vocabulary from grants_gov.csv and takes the mean (possibly not the best way to do this).

In [19]:
faculty_profiles = defaultdict(list)
for index, row in grant_history_df.iterrows():
    faculty_profiles[row["PI Name"]].append(row["Title"])

In [20]:
faculty_vectorizer = TfidfVectorizer(vocabulary=grants_vectorizer.vocabulary_)
for faculty in faculty_profiles:
    research = faculty_profiles[faculty]
    vectorized = faculty_vectorizer.fit_transform(research)
    faculty_profiles[faculty] = np.mean(vectorized, axis=0)

In [21]:
faculty_list = list(faculty_profiles.keys())
print("Total number of faculty: " + str(len(faculty_profiles)))

Total number of faculty: 2109


Match faculty to grants by choosing the top five grants with the highest dot product similarity. Requires faculty and grants to share the same features

In [22]:
top_matches = {}
test_faculty = random.sample(faculty_list, 10)
for faculty in test_faculty:
    profile = faculty_profiles[faculty]
    matches = []
    for j in range(grants_matrix.shape[0]):
        grant = grants_matrix[j]
        matches.append((np.dot(grant.toarray(), profile.T).item(0), j))
    top_matches[faculty] = sorted(matches)[-5:]

matches = defaultdict(list)
matches_similarities = defaultdict(list)
for faculty in top_matches:
    for similarity, index in top_matches[faculty]:
        matches[faculty].append(grants_df[index:index+1])
        matches_similarities[faculty].append(similarity)

In [24]:
# Grant matches for first faculty in top_matches for grants_gov
faculty = list(matches.keys())[0]
print(faculty)
for match in matches[faculty]:
    display(match)

Dandelet, Thomas J


,OpportunityID,OpportunityTitle,OpportunityNumber,OpportunityCategory,FundingInstrumentType,CategoryOfFundingActivity,CategoryExplanation,CFDANumbers,EligibleApplicants,AdditionalInformationOnEligibility,AgencyCode,AgencyName,PostDate,CloseDate,LastUpdatedDate,AwardCeiling,AwardFloor,EstimatedTotalProgramFunding,ExpectedNumberOfAwards,Description,Version,CostSharingOrMatchingRequirement,ArchiveDate,GrantorContactEmail,GrantorContactEmailDescription,GrantorContactText,AdditionalInformationURL,AdditionalInformationText,OpportunityCategoryExplanation,CloseDateExplanation
26487,45177,State MCH Early Childhood Comprehensive System...,HRSA-09-176,D,G,HL,NaN,93.110,25,Eligibility for this funding opportunity is li...,HHS-HRSA,Health Resources and Services Administration,2052009,3112009.0,2102009,140000.0,100000.0,7280000.0,52.0,The purpose of the State Early Childhood Compr...,Synopsis 7,No,4172009.0,CallCenter@HRSA.GOV,Contact HRSA Call Center at 877-Go4-HRSA (877-...,CallCenter@HRSA.GOV&lt;br/&gt;CallCenter@HRSA....,https://grants.hrsa.gov/webExternal/SFO.asp?ID...,https://grants.hrsa.gov/webExternal/SFO.asp?ID...,NaN,NaN


,OpportunityID,OpportunityTitle,OpportunityNumber,OpportunityCategory,FundingInstrumentType,CategoryOfFundingActivity,CategoryExplanation,CFDANumbers,EligibleApplicants,AdditionalInformationOnEligibility,AgencyCode,AgencyName,PostDate,CloseDate,LastUpdatedDate,AwardCeiling,AwardFloor,EstimatedTotalProgramFunding,ExpectedNumberOfAwards,Description,Version,CostSharingOrMatchingRequirement,ArchiveDate,GrantorContactEmail,GrantorContactEmailDescription,GrantorContactText,AdditionalInformationURL,AdditionalInformationText,OpportunityCategoryExplanation,CloseDateExplanation
4725,275503,BLM BiState Sage Grouse Habitat Restoration an...,L15AS00047,D,CA,NR,NaN,15.231,99,NaN,DOI-BLM,Bureau of Land Management,3282015,5292015.0,3282015,1500000.0,10000.0,3000000.0,2.0,This program seeks to partner with the state w...,Synopsis 1,No,7292015.0,skaller@blm.gov,skaller@blm.gov,"Susan Kaller, Grants and Agreements Specialist...",http://www.grants.gov,Click on Related Documents tab to access Full ...,NaN,NaN


,OpportunityID,OpportunityTitle,OpportunityNumber,OpportunityCategory,FundingInstrumentType,CategoryOfFundingActivity,CategoryExplanation,CFDANumbers,EligibleApplicants,AdditionalInformationOnEligibility,AgencyCode,AgencyName,PostDate,CloseDate,LastUpdatedDate,AwardCeiling,AwardFloor,EstimatedTotalProgramFunding,ExpectedNumberOfAwards,Description,Version,CostSharingOrMatchingRequirement,ArchiveDate,GrantorContactEmail,GrantorContactEmailDescription,GrantorContactText,AdditionalInformationURL,AdditionalInformationText,OpportunityCategoryExplanation,CloseDateExplanation
17868,291146,Leading English Education Programs in Kyushu/Y...,PAS-FUKUOKA-FY2017-0001,D,"G,O","ED,O",U.S.-Japan bilateral relationship,19.040,"01,06,12,13,20,21,23",The U.S. Consulate Fukuoka Public Affairs Sect...,DOS,Department of State,1112017,3152017.0,1112017,1000.0,500.0,1000.0,5.0,The U.S. Consulate Fukuoka Public Affairs Sect...,Synopsis 1,No,4142017.0,FACRS@state.gov,Fukuoka PAS Grants Management,Fukuoka PAS &lt;br/&gt;Grants Management &lt;b...,NaN,NaN,NaN,NaN


,OpportunityID,OpportunityTitle,OpportunityNumber,OpportunityCategory,FundingInstrumentType,CategoryOfFundingActivity,CategoryExplanation,CFDANumbers,EligibleApplicants,AdditionalInformationOnEligibility,AgencyCode,AgencyName,PostDate,CloseDate,LastUpdatedDate,AwardCeiling,AwardFloor,EstimatedTotalProgramFunding,ExpectedNumberOfAwards,Description,Version,CostSharingOrMatchingRequirement,ArchiveDate,GrantorContactEmail,GrantorContactEmailDescription,GrantorContactText,AdditionalInformationURL,AdditionalInformationText,OpportunityCategoryExplanation,CloseDateExplanation
16919,289931,Small Grants Program,PAS-TOKYO-FY2017-0001,D,"G,O","AR,ED,EN,ENV,FN,IS,LJL,O",U.S. &#8211; JAPAN BILATERAL RELATIONSHIP REG...,19.040,"01,02,05,06,12,13,20,21",The U.S. Embassy Tokyo Public Affairs Section ...,DOS,Department of State,11072016,9302017.0,11062016,100000.0,1000.0,100000.0,100.0,The U.S. Embassy Tokyo Public Affairs Section ...,Synopsis 1,No,10302017.0,TokyoPASGrants@state.gov,Tokyo PAS Grants Management,Tokyo PAS &lt;br/&gt;Grants Management &lt;br/...,NaN,NaN,NaN,NaN


,OpportunityID,OpportunityTitle,OpportunityNumber,OpportunityCategory,FundingInstrumentType,CategoryOfFundingActivity,CategoryExplanation,CFDANumbers,EligibleApplicants,AdditionalInformationOnEligibility,AgencyCode,AgencyName,PostDate,CloseDate,LastUpdatedDate,AwardCeiling,AwardFloor,EstimatedTotalProgramFunding,ExpectedNumberOfAwards,Description,Version,CostSharingOrMatchingRequirement,ArchiveDate,GrantorContactEmail,GrantorContactEmailDescription,GrantorContactText,AdditionalInformationURL,AdditionalInformationText,OpportunityCategoryExplanation,CloseDateExplanation
35048,55169,Legacy Resource Management Program,W9132T-10-LEGACYRFP,D,CA,ENV,Conservation and Cultural Activities,NaN,99,NaN,DOD-COE,Dept. of the Army -- Corps of Engineers,6112010,7162010.0,6112010,NaN,NaN,NaN,NaN,REQUEST FOR PRE-PROPOSALS FOR FY 2011 LEGACY...,Synopsis 1,No,8152010.0,Peter.Boice@osd.mil,Legacy Program Director,"L. Peter Boice &lt;br/&gt;Deputy Director, Nat...",https://www.dodlegacy.org/Legacy/intro/about.a...,DoD Legacy Website,NaN,NaN


In [25]:
# Match percentages out of maximum
max_similarity = max(matches_similarities[faculty])
for similarity in matches_similarities[faculty]:
    print(similarity/max_similarity)

0.8917975610697346
0.8982302253779051
0.9112710485516938
0.9296661739782758
1.0


<h3>Testing with NSF grants</h3>

In [26]:
nsf_top_matches = {}
nsf_test_faculty = random.sample(faculty_list, 10)
for faculty in nsf_test_faculty:
    profile = faculty_profiles[faculty]
    matches = []
    for j in range(nsf_matrix.shape[0]):
        grant = nsf_matrix[j]
        matches.append((np.dot(grant.toarray(), profile.T).item(0), j))
    nsf_top_matches[faculty] = sorted(matches)[-5:]
    
nsf_matches = defaultdict(list)
nsf_matches_similarities = defaultdict(list)
for faculty in nsf_top_matches:
    for similarity, index in nsf_top_matches[faculty]:
        nsf_matches[faculty].append(nsf_grants_df[index:index+1])
        nsf_matches_similarities[faculty].append(similarity)

In [27]:
# Grant matches for first faculty in nsf_top_matches for NSF grants
faculty_nsf = list(nsf_matches.keys())[0]
print(faculty_nsf)
for match in nsf_matches[faculty_nsf]:
    display(match)

Moskowitz, Joel M


,Headline,Due Date Start,Due Date End,Description,Link
3,Industry-University Cooperative Research Cente...,None,2017-02-28,The Industry-University Cooperative Research C...,https://nsf.gov/funding/pgm_summ.jsp?pims_id=5...


,Headline,Due Date Start,Due Date End,Description,Link
33,Industry-University Cooperative Research Cente...,None,2017-02-28,The Industry-University Cooperative Research C...,https://nsf.gov/funding/pgm_summ.jsp?pims_id=5...


,Headline,Due Date Start,Due Date End,Description,Link
48,Industry-University Cooperative Research Cente...,None,2017-02-28,The Industry-University Cooperative Research C...,https://nsf.gov/funding/pgm_summ.jsp?pims_id=5...


,Headline,Due Date Start,Due Date End,Description,Link
146,Industry-University Cooperative Research Cente...,None,2017-02-28,The Industry-University Cooperative Research C...,https://nsf.gov/funding/pgm_summ.jsp?pims_id=5...


,Headline,Due Date Start,Due Date End,Description,Link
241,Industry-University Cooperative Research Cente...,None,2017-02-28,The Industry-University Cooperative Research C...,https://nsf.gov/funding/pgm_summ.jsp?pims_id=5...


In [28]:
# Links to the matches
pd.options.display.max_colwidth = 100
for match in nsf_matches[faculty_nsf]:
    print(match['Link'].to_string())

3    https://nsf.gov/funding/pgm_summ.jsp?pims_id=5501&org=NSF&sel_org=NSF&from=fund
33    https://nsf.gov/funding/pgm_summ.jsp?pims_id=5501&org=NSF&sel_org=NSF&from=fund
48    https://nsf.gov/funding/pgm_summ.jsp?pims_id=5501&org=NSF&sel_org=NSF&from=fund
146    https://nsf.gov/funding/pgm_summ.jsp?pims_id=5501&org=NSF&sel_org=NSF&from=fund
241    https://nsf.gov/funding/pgm_summ.jsp?pims_id=5501&org=NSF&sel_org=NSF&from=fund


In [29]:
# Match percentages for NSF grants out of maximum
nsf_max_similarity = max(nsf_matches_similarities[faculty_nsf])
for similarity in nsf_matches_similarities[faculty_nsf]:
    print(similarity/nsf_max_similarity)

1.0
1.0
1.0
1.0
1.0


<h1> Clustering approach </h1>

Use random set of training grants from grants.gov grants and assigns them to professors using one of the three approaches below.
Then takes a test sample from NSF grants and see which professors are matched with it.

<b> Only run one of the three cells below </b>

In [30]:
# This grant assignment approach assigns professors the grants that they have the highest similiarity with.
# Multiple professors may be assigned to a single grant.

training_size = 1000
training_grants = sklearn.utils.shuffle(grants_matrix[:])[:training_size]
progress = 0
faculty_to_grants = {}
for faculty in faculty_profiles:
    profile = faculty_profiles[faculty]
    matches = []
    for j in range(training_grants.shape[0]):
        grant = training_grants[j]
        matches.append((np.dot(grant.toarray(), profile.T), j))
    faculty_to_grants[faculty] = [x[1] for x in sorted(matches)[-5:]]
    if progress % 250 == 0:
        print(str(progress) + "/" + str(len(faculty_profiles)))
    progress += 1
print(str(len(faculty_profiles)) + "/" + str(len(faculty_profiles)))

0/2109
250/2109
500/2109
750/2109
1000/2109
1250/2109
1500/2109
1750/2109
2000/2109
2109/2109


In [ ]:
# This grant assignment approach assigns professors the grants that they have the highest similiarity with.
# One-to-one matching with grants and professors.
# If using this matching approach, a faculty may appear muliple times when testing matches. Maybe this can be used
# as a sign that this faculty is an exceptionally good match?

training_size = 1000
training_grants = sklearn.utils.shuffle(grants_matrix[:])[:training_size]
progress = 0
faculty_to_grants = defaultdict(list)
for i in range(training_grants.shape[0]):
    grant = training_grants[i]
    top_faculty = None
    top_score = 0
    for faculty in faculty_profiles:
        profile = faculty_profiles[faculty]
        score = np.dot(grant.toarray(), profile.T)
        if score > top_score:
            top_faculty = faculty
            top_score = score
    faculty_to_grants[top_faculty].append(i)
    if progress % 250 == 0:
        print(str(progress) + "/" + str(training_size))
    progress += 1
print(str(training_size) + "/" + str(training_size))

In [ ]:
# Same approach as above, but with cosine similarity to determine matching between grant and faculty.
# About 10x slower than above

training_size = 1000
training_grants = sklearn.utils.shuffle(grants_matrix[:])[:training_size]
progress = 0
faculty_to_grants = defaultdict(list)
for i in range(training_grants.shape[0]):
    grant = training_grants[i]
    top_faculty = None
    top_score = 0
    for faculty in faculty_profiles:
        profile = faculty_profiles[faculty]
        score = cosine_similarity(grant.toarray(), profile).item(0)
        if score > top_score:
            top_faculty = faculty
            top_score = score
    faculty_to_grants[top_faculty].append(i)
    if progress % 250 == 0:
        print(str(progress) + "/" + str(training_size))
    progress += 1
print(str(training_size) + "/" + str(training_size))

Takes a random NSF grant and matches it with faculty.
Prints the test NSF grant information, link to the grant, and the faculty that it matched with.

In [31]:
# Take a random NSF grant and match with professors
random_index = random.sample((range(nsf_matrix.shape[0])), 1)[0]
test_grant = nsf_matrix[random_index]
closest_grants = []
for index in range(training_grants.shape[0]):
    training_grant = training_grants[index]
    # Uses dot similarity
    closest_grants.append((np.dot(training_grant, test_grant.T).toarray()[0], index))
    
# Finds the 10 grants that are closest to the test grant
closest_grants = [x[1] for x in sorted(closest_grants)[-5:]]

clustering_matches = []
for faculty in faculty_to_grants:
    faculty_grants = faculty_to_grants[faculty]
    for closest_grant in closest_grants:
        if closest_grant in faculty_grants:
            clustering_matches.append(faculty)

# Print out test grant, link to grant, and matched faculty
test_grant_full = nsf_grants_df[random_index:random_index+1]
display(test_grant_full)
for faculty in clustering_matches:
    print(faculty)
    
print(test_grant_full['Link'].to_string())

,Headline,Due Date Start,Due Date End,Description,Link
117,"Mind, Machine and Motor Nexus",2017-09-01,2017-09-15,"The Mind, Machine and Motor Nexus (M3X) program supports fundamental research at the intersectio...",https://nsf.gov/funding/pgm_summ.jsp?pims_id=505402&org=NSF&sel_org=NSF&from=fund


Skabardonis, Alexander
Harvey, Allison
Full, Robert J
Givental, Alexander B
Givental, Alexander B
Agogino, Alice M
Chow, Fotini K
Suzuki, Yuri
Whaley, K Birgitta
Tomizuka, Masayoshi
Engel, Jerome
Varaiya, Pravin P
Poo, Mu-Ming
Chang-Hasnain, Constance J
Cassidy, Michael J
Clarke, John
Nomura, Yasunori
Alivisatos, A Paul
Chorin, Alexandre J
Dibble, Robert W
Kammen, Daniel M
Dynes, Robert
Farrell, Alexander E
Wirth, Brian
Oren, Shmuel S
Vazirani, Umesh V
Morris, John W
Morris, John W
Chua, Leon O
Sachs, Rainer K
Miller, William H
Chaston, Christopher
Grossman, Jeffrey
Krucker, Sam
Iglesia, Enrique
Horowitz, Roberto
Frenklach, Michael Y
Schor, Clifton M
Kazerooni, Homayoon
Marchis, Franck
Silverstein, Gordon
Subramanian, Vivek
Javey, Ali
Peterson, Per F
Lipman, Timothy E
Niknejad, Ali M
Sangiovanni-Vincentelli, Alberto
Ramchandran, Kannan
Katz, Randy H
Nikolic, Borivoje
Brillinger, David R
Smale, Stephen
Frenkel, Edward
Auslander, David M
Grigoropoulos, Costas P
Shaheen, Susan A
Sanders, 